## SSH key pair generation

we create a SSH key pair that will be used to connect to the deployed node.

In [1]:
%x(ssh-keygen -b 2048 -t rsa -f /tmp/sshkey -q -N "")

""

## Job and deployment definition

In [2]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
  :resources => ["{cluster='hercule'}/nodes=4,{cluster='sagittaire'}/nodes=4, walltime=1:00:00"],
    :site      => 'lyon',
    :types     => ["deploy"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 8})],
    :command   => "sleep 86400"
    })
myxp.define_deployment({
    :site           => 'lyon',
    :environment    => "jessie-x64-nfs",
    :roles          => %w{ mynode },
    :key            => File.read("/tmp/sshkey.pub")
})

[{:site=>"lyon", :environment=>"jessie-x64-nfs", :roles=>["mynode"], :key=>"ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC0tePmcwAkH7XU/KKTfakT8G8ToRaxSpO3/NVRjPSLCBzxDY9TVKytJ+4VtN2j00ODy9L5/8ZCMwu6iW7VW8ET9WKq7ZMaVPxYgB2QG2mjxrKQLCgHrP/IYd2wANK6R0ZLssfDgHeXs0Hs8LuVbnFI00qcRAA1katNy6Enf5H5nsU3BzUsl2rnNEgkmkBDJM3T1STbM/pXxwLtYLSViVEtWjB6zWGI12Jk1+uv6Z/aHed17f8RhrdSNlhMv7aHOmEW/VFwnpE2ENyhqGCdZcgh0I8f/SimeuOnGdSPD4gw1Coyj5X07DB6HMXvHR3qsMnIidHSL5RJ7FYRhxTZY3yF root@ec3738337ee4\n", :jobs=>[]}]

## Job and deployment submission

In [3]:
myxp.submit
myxp.wait_for_jobs
myxp.deploy

...................... [OK]
Summary of the deployment
------------------------------------------------------------
                Name            Deployed          Undeployed

------------------------------------------------------------
              mynode                   8                   0



{"mynode"=>["hercule-1.lyon.grid5000.fr", "hercule-2.lyon.grid5000.fr", "hercule-3.lyon.grid5000.fr", "hercule-4.lyon.grid5000.fr", "sagittaire-31.lyon.grid5000.fr", "sagittaire-42.lyon.grid5000.fr", "sagittaire-57.lyon.grid5000.fr", "sagittaire-69.lyon.grid5000.fr"]}

## Install fping on all the nodes

In [4]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = ['apt-get update -qq && apt-get install -qy fping']
end

Connected to hercule-3.lyon.grid5000.fr...
Connected to hercule-1.lyon.grid5000.fr...
Connected to hercule-2.lyon.grid5000.fr...
Connected to sagittaire-31.lyon.grid5000.fr...
Connected to sagittaire-42.lyon.grid5000.fr...
Connected to sagittaire-57.lyon.grid5000.fr...
Connected to sagittaire-69.lyon.grid5000.fr...
Connected to hercule-4.lyon.grid5000.fr...
[command][hercule-1.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][hercule-2.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][hercule-3.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][hercule-4.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][sagittaire-31.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][sagittaire-42.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][sagittaire-57.lyon.grid5000.fr] apt-get update -qq && apt-get install -qy fping
[command][sagittaire-69.ly

{"hercule-1.lyon.grid5000.fr"=>"Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following NEW packages will be installed:\n  fping\n0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.\nNeed to get 37.4 kB of archives.\nAfter this operation, 133 kB of additional disk space will be used.\nGet:1 http://ftp.debian.org/debian/ jessie/main fping amd64 3.10-2 [37.4 kB]\ndebconf: unable to initialize frontend: Dialog\ndebconf: (TERM is not set, so the dialog frontend is not usable.)\ndebconf: falling back to frontend: Readline\ndebconf: unable to initialize frontend: Readline\ndebconf: (This frontend requires a controlling tty.)\ndebconf: falling back to frontend: Teletype\ndpkg-preconfigure: unable to re-open stdin: \nFetched 37.4 kB in 0s (1,127 kB/s)\nSelecting previously unselected package fping.\r\n(Reading database ... \r(Reading database ... 5%\r(Reading database ... 10%\r(Reading database ... 15%\r(Reading database ... 20%\r(Reading 

## Each node pings all nodes

In [5]:
servers = roles('mynode')
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = "cat > /tmp/hosts << EOF
#{servers.join("\n")}
EOF"
end

Connected to hercule-1.lyon.grid5000.fr...
Connected to hercule-2.lyon.grid5000.fr...
Connected to sagittaire-31.lyon.grid5000.fr...
Connected to sagittaire-57.lyon.grid5000.fr...
Connected to hercule-3.lyon.grid5000.fr...
Connected to hercule-4.lyon.grid5000.fr...
Connected to sagittaire-42.lyon.grid5000.fr...
Connected to sagittaire-69.lyon.grid5000.fr...
[command][hercule-1.lyon.grid5000.fr] cat > /tmp/hosts << EOF
hercule-1.lyon.grid5000.fr
hercule-2.lyon.grid5000.fr
hercule-3.lyon.grid5000.fr
hercule-4.lyon.grid5000.fr
sagittaire-31.lyon.grid5000.fr
sagittaire-42.lyon.grid5000.fr
sagittaire-57.lyon.grid5000.fr
sagittaire-69.lyon.grid5000.fr
EOF
[command][hercule-2.lyon.grid5000.fr] cat > /tmp/hosts << EOF
hercule-1.lyon.grid5000.fr
hercule-2.lyon.grid5000.fr
hercule-3.lyon.grid5000.fr
hercule-4.lyon.grid5000.fr
sagittaire-31.lyon.grid5000.fr
sagittaire-42.lyon.grid5000.fr
sagittaire-57.lyon.grid5000.fr
sagittaire-69.lyon.grid5000.fr
EOF
[command][hercule-3.lyon.grid5000.fr] cat > 

{"hercule-1.lyon.grid5000.fr"=>"", "hercule-2.lyon.grid5000.fr"=>"", "hercule-3.lyon.grid5000.fr"=>"", "hercule-4.lyon.grid5000.fr"=>"", "sagittaire-31.lyon.grid5000.fr"=>"", "sagittaire-42.lyon.grid5000.fr"=>"", "sagittaire-57.lyon.grid5000.fr"=>"", "sagittaire-69.lyon.grid5000.fr"=>""}

In [31]:
n = 100
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = "fping -C #{n} -q -s -e -f /tmp/hosts"
end

Connected to hercule-1.lyon.grid5000.fr...
Connected to hercule-2.lyon.grid5000.fr...
Connected to hercule-3.lyon.grid5000.fr...
Connected to sagittaire-31.lyon.grid5000.fr...
Connected to sagittaire-42.lyon.grid5000.fr...
Connected to sagittaire-57.lyon.grid5000.fr...
Connected to hercule-4.lyon.grid5000.fr...
Connected to sagittaire-69.lyon.grid5000.fr...
[command][hercule-1.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][hercule-2.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][hercule-3.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][hercule-4.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][sagittaire-31.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][sagittaire-42.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][sagittaire-57.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[command][sagittaire-69.lyon.grid5000.fr] fping -C 100 -q -s -e -f /tmp/hosts
[stderr][hercule-1.lyon.grid5000

{"hercule-1.lyon.grid5000.fr"=>"hercule-1.lyon.grid5000.fr     : 0.01 0.01 0.00 0.01 0.01 0.01 0.00 0.01 0.00 0.00 0.00 0.01 0.01 0.00 0.00 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.01 0.00 0.01 0.01 0.01 0.00 0.00 0.01 0.00 0.01 0.01 0.01 0.01 0.00 0.00 0.01 0.01 0.00 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 0.01 0.01 0.01 0.03 0.01 0.01 0.01 0.01 0.00 0.00 0.01 0.00 0.01 0.00 0.01 0.01 0.00 0.00 0.01 0.01 0.00 0.01 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.01 0.01 0.01 0.01 0.00\nhercule-2.lyon.grid5000.fr     : 0.04 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.04 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0

In [46]:
hh = []
h.each do |k,v|
    hh.concat(h[k]
      .split("\n")[0..servers.size - 1]
        .map{|x| x.split(" ")}
        .map{|a| [k, a[0], a[2..-1].map{|l| l.to_f/n}.sum]})
end
hh.size

64

In [55]:
node_to_key = h.keys.sort.each_with_index.inject([]){ |h,i| h<<i}.to_h
key_to_node = node_to_key.map{|k,v| [v,k]}.to_h

{0=>"hercule-1.lyon.grid5000.fr", 1=>"hercule-2.lyon.grid5000.fr", 2=>"hercule-3.lyon.grid5000.fr", 3=>"hercule-4.lyon.grid5000.fr", 4=>"sagittaire-31.lyon.grid5000.fr", 5=>"sagittaire-42.lyon.grid5000.fr", 6=>"sagittaire-57.lyon.grid5000.fr", 7=>"sagittaire-69.lyon.grid5000.fr"}

In [56]:
x=hh.map{|x| node_to_key[x[0]]}
y=hh.map{|x| node_to_key[x[1]]}
fill=hh.map{|x|x[2]}

[0.005400000000000001, 0.030200000000000077, 0.03050000000000008, 0.030000000000000075, 0.07100000000000002, 0.07260000000000004, 0.071, 0.07160000000000002, 0.05180000000000002, 0.0010000000000000002, 0.03100000000000005, 0.030000000000000075, 0.0716, 0.07300000000000005, 0.07070000000000001, 0.07130000000000002, 0.08089999999999996, 0.08029999999999994, 0.08069999999999995, 0.08019999999999995, 0.10010000000000008, 0.02019999999999999, 0.10010000000000008, 0.10010000000000008, 0.0538000000000001, 0.03080000000000006, 0.0010000000000000002, 0.030000000000000075, 0.07150000000000002, 0.07340000000000005, 0.07130000000000002, 0.07200000000000004, 0.030200000000000077, 0.030100000000000078, 0.030100000000000078, 0.0001, 0.07129999999999999, 0.07200000000000004, 0.07050000000000001, 0.0713, 0.08039999999999996, 0.08029999999999995, 0.08029999999999995, 0.07999999999999996, 0.02009999999999999, 0.10000000000000007, 0.10030000000000008, 0.10040000000000009, 0.08049999999999995, 0.0801999999

In [57]:
require 'nyaplot'
plot5 = Nyaplot::Plot.new
hm = plot5.add(:heatmap, x, y, fill)
hm.stroke_color("#fff")
hm.stroke_width("0")
hm.width(1)
hm.height(1)
plot5.legend(true)
plot5.show

#<Nyaplot::Frame:0x00000002a11c68 @properties={:panes=>[#<Nyaplot::Plot:0x00000002ef6e68 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000002a12780 @properties={:type=>:heatmap, :options=>{:x=>"data0", :y=>"data1", :fill=>"data2", :stroke_color=>"#fff", :stroke_width=>"0", :width=>1, :height=>1}, :data=>"4fd4c80c-9368-4546-a465-2819b46bf58b"}, @xrange=[0, 7], @yrange=[0, 7]>], :options=>{:legend=>true, :zoom=>true, :width=>800, :xrange=>[0, 7], :yrange=>[0, 7]}}>], :data=>{"4fd4c80c-9368-4546-a465-2819b46bf58b"=>#<Nyaplot::DataFrame:0x00000002ef6490 @name="4fd4c80c-9368-4546-a465-2819b46bf58b", @rows=[{:data0=>0, :data1=>0, :data2=>0.005400000000000001}, {:data0=>0, :data1=>1, :data2=>0.030200000000000077}, {:data0=>0, :data1=>2, :data2=>0.03050000000000008}, {:data0=>0, :data1=>3, :data2=>0.030000000000000075}, {:data0=>0, :data1=>4, :data2=>0.07100000000000002}, {:data0=>0, :data1=>5, :data2=>0.07260000000000004}, {:data0=>0, :data1=>6, :data2=>0.071}, {:data0=>0, :data1=>7, :data2=>0.07160000000000002}, {:data0=>1, :data1=>0, :data2=>0.05180000000000002}, {:data0=>1, :data1=>1, :data2=>0.0010000000000000002}, {:data0=>1, :data1=>2, :data2=>0.03100000000000005}, {:data0=>1, :data1=>3, :data2=>0.030000000000000075}, {:data0=>1, :data1=>4, :data2=>0.0716}, {:data0=>1, :data1=>5, :data2=>0.07300000000000005}, {:data0=>1, :data1=>6, :data2=>0.07070000000000001}, {:data0=>1, :data1=>7, :data2=>0.07130000000000002}, {:data0=>5, :data1=>0, :data2=>0.08089999999999996}, {:data0=>5, :data1=>1, :data2=>0.08029999999999994}, {:data0=>5, :data1=>2, :data2=>0.08069999999999995}, {:data0=>5, :data1=>3, :data2=>0.08019999999999995}, {:data0=>5, :data1=>4, :data2=>0.10010000000000008}, {:data0=>5, :data1=>5, :data2=>0.02019999999999999}, {:data0=>5, :data1=>6, :data2=>0.10010000000000008}, {:data0=>5, :data1=>7, :data2=>0.10010000000000008}, {:data0=>2, :data1=>0, :data2=>0.0538000000000001}, {:data0=>2, :data1=>1, :data2=>0.03080000000000006}, {:data0=>2, :data1=>2, :data2=>0.0010000000000000002}, {:data0=>2, :data1=>3, :data2=>0.030000000000000075}, {:data0=>2, :data1=>4, :data2=>0.07150000000000002}, {:data0=>2, :data1=>5, :data2=>0.07340000000000005}, {:data0=>2, :data1=>6, :data2=>0.07130000000000002}, {:data0=>2, :data1=>7, :data2=>0.07200000000000004}, {:data0=>3, :data1=>0, :data2=>0.030200000000000077}, {:data0=>3, :data1=>1, :data2=>0.030100000000000078}, {:data0=>3, :data1=>2, :data2=>0.030100000000000078}, {:data0=>3, :data1=>3, :data2=>0.0001}, {:data0=>3, :data1=>4, :data2=>0.07129999999999999}, {:data0=>3, :data1=>5, :data2=>0.07200000000000004}, {:data0=>3, :data1=>6, :data2=>0.07050000000000001}, {:data0=>3, :data1=>7, :data2=>0.0713}, {:data0=>4, :data1=>0, :data2=>0.08039999999999996}, {:data0=>4, :data1=>1, :data2=>0.08029999999999995}, {:data0=>4, :data1=>2, :data2=>0.08029999999999995}, {:data0=>4, :data1=>3, :data2=>0.07999999999999996}, {:data0=>4, :data1=>4, :data2=>0.02009999999999999}, {:data0=>4, :data1=>5, :data2=>0.10000000000000007}, {:data0=>4, :data1=>6, :data2=>0.10030000000000008}, {:data0=>4, :data1=>7, :data2=>0.10040000000000009}, {:data0=>6, :data1=>0, :data2=>0.08049999999999995}, {:data0=>6, :data1=>1, :data2=>0.08019999999999995}, {:data0=>6, :data1=>2, :data2=>0.08039999999999996}, {:data0=>6, :data1=>3, :data2=>0.08009999999999995}, {:data0=>6, :data1=>4, :data2=>0.10010000000000008}, {:data0=>6, :data1=>5, :data2=>0.10010000000000006}, {:data0=>6, :data1=>6, :data2=>0.020299999999999988}, {:data0=>6, :data1=>7, :data2=>0.10020000000000008}, {:data0=>7, :data1=>0, :data2=>0.08099999999999996}, {:data0=>7, :data1=>1, :data2=>0.08069999999999995}, {:data0=>7, :data1=>2, :data2=>0.08019999999999995}, {:data0=>7, :data1=>3, :data2=>0.08009999999999995}, {:data0=>7, :data1=>4, :data2=>0.10000000000000007}, {:data0=>7, :data1=>5, :data2=>0.10020000000000008}, {:data0=>7, :data1=>6, :data2=>0.10010000000000008}, {:data0=>7, :data1=>7, :data2=>0.020099999999999986}]>}, :extension=>[]}>

["display_data", "<IDS|MSG>", "15d21297291a6db81806f2610eda85e8227fb57fa674f1255f1eecbb47f8e6da", "{\"msg_type\":\"display_data\",\"msg_id\":\"672643d3-2761-47e3-9fd2-72b60eaff86d\",\"username\":\"kernel\",\"session\":\"16d1a89d-4e55-4ab3-8d8d-bbce7f5deac8\",\"version\":\"5.0\"}", "{\"username\":\"username\",\"version\":\"5.0\",\"msg_id\":\"2F7142F270C7430BA9E4DEA025DF8C2D\",\"msg_type\":\"execute_request\",\"session\":\"60252D33CCA34075B2EE3E0C09154D21\"}", "{}", "{\"data\":{\"text/html\":\"<div id='vis-462ae272-7493-4232-a5f3-71475b45b99e'></div>\\n<script>\\n(function(){\\n    var render = function(){\\n        var model = {\\\"panes\\\":[{\\\"diagrams\\\":[{\\\"type\\\":\\\"heatmap\\\",\\\"options\\\":{\\\"x\\\":\\\"data0\\\",\\\"y\\\":\\\"data1\\\",\\\"fill\\\":\\\"data2\\\",\\\"stroke_color\\\":\\\"#fff\\\",\\\"stroke_width\\\":\\\"0\\\",\\\"width\\\":1,\\\"height\\\":1},\\\"data\\\":\\\"4fd4c80c-9368-4546-a465-2819b46bf58b\\\"}],\\\"options\\\":{\\\"legend\\\":true,\\\"zoom\\\":

In [ ]:
myxp.clean